In [5]:
!ls /content/drive

'My Drive'


In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn import preprocessing, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
np.random.seed(1)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!ls /content/drive/My\ Drive

 Brad_work.ipynb		    hw2
'Colab Notebooks'		    Individual_Component.zip
 COMP9021_Lec1_wyn_note.ipynb	    test.csv
'Copy of new.drawio'		    training.csv
'Copy of Untitled Diagram.drawio'   Untitled0.ipynb
'CSESoc FYG 2019.gdoc'		   'Untitled Diagram (1).drawio'
'CSESoc FYG 2019.pdf'		   'Untitled Diagram.drawio'
 Group_Component.zip		   'Untitled document.gdoc'
 GSOE9820.ipynb


In [0]:
test_df = pd.read_csv("/content/drive/My Drive/test.csv")

In [10]:
test_df

,article_number,article_words,topic
0,9501,"world,complet,pharmaceut,tianjin,tianjin,chin,...",IRRELEVANT
1,9502,"copy,sunday,weekend,ec,friday,eu,includ,limit,...",IRRELEVANT
2,9503,"heavy,heavy,gabriel,morn,morn,equit,cent,cent,...",FOREX MARKETS
3,9504,"research,jess,hit,anticip,comput,comput,comput...",IRRELEVANT
4,9505,"provid,provid,luxembourg,court,court,case,opin...",IRRELEVANT
...,...,...,...
495,9996,"symantec,soon,interfac,provid,provid,comput,co...",IRRELEVANT
496,9997,"hit,hit,hit,hit,hit,hit,hit,hit,hit,beat,beat,...",SPORTS
497,9998,"cent,cent,cent,cent,match,declin,act,rate,rate...",MONEY MARKETS
498,9999,"cnmv,cnmv,stock,count,count,week,group,friday,...",SHARE LISTINGS


In [0]:
train_df = pd.read_csv("/content/drive/My Drive/training.csv")

In [12]:
train_df

,article_number,article_words,topic
0,1,"open,absent,cent,cent,cent,stock,inflow,rate,k...",FOREX MARKETS
1,2,"morn,stead,end,end,day,day,day,patch,patch,pat...",MONEY MARKETS
2,3,"socc,socc,world,world,recent,law,fifa,fifa,fif...",SPORTS
3,4,"open,forint,forint,forint,forint,cent,cent,ste...",FOREX MARKETS
4,5,"morn,complet,weekend,minut,minut,minut,arrow,d...",IRRELEVANT
...,...,...,...
9495,9496,"cloud,provid,hope,centur,erupt,rule,recent,sou...",DEFENCE
9496,9497,"stock,stock,stock,declin,access,week,worry,blo...",IRRELEVANT
9497,9498,"rate,million,million,belarus,dollar,dollar,nov...",FOREX MARKETS
9498,9499,"flow,bullet,bullet,bullet,bullet,bullet,bullet...",IRRELEVANT


# Preprocessing
*base on the work of Raymond*

Use  ``` CountVectorizer ``` and ``` TfidfVectorizer ``` to extract features


In [13]:
train_x = train_df.article_words
train_y = train_df.topic
test_x = test_df.article_words
test_y = test_df.topic

en_train_y = preprocessing.LabelEncoder().fit_transform(train_y)
en_test_y = preprocessing.LabelEncoder().fit_transform(test_y)
categories=list(set(train_df['topic']))
print(categories)

['BIOGRAPHIES PERSONALITIES PEOPLE', 'SHARE LISTINGS', 'IRRELEVANT', 'FOREX MARKETS', 'HEALTH', 'ARTS CULTURE ENTERTAINMENT', 'DOMESTIC MARKETS', 'MONEY MARKETS', 'SPORTS', 'SCIENCE AND TECHNOLOGY', 'DEFENCE']


In [0]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    sp = tf.SparseTensor(indices, coo.data, coo.shape)
    sp = tf.sparse.reorder(sp)
    return sp

In [0]:
# get Count vectors
vectorizer = CountVectorizer().fit(train_x)
train_count_csr = vectorizer.transform(train_x)
test_count_csr = vectorizer.transform(test_x)

# change count vectors from scipy.sparse.csr.csr_matrix to tf.sparse.SparseTensor
train_count_st = convert_sparse_matrix_to_sparse_tensor(train_count_csr)
test_count_st = convert_sparse_matrix_to_sparse_tensor(test_count_csr)

# change count vectors from tf.sparse.SparseTensor to tf.Tensor
train_count_dense = tf.sparse.to_dense(train_count_st)
test_count_dense = tf.sparse.to_dense(test_count_st)

# get Tfid vectors
vectorizer = TfidfVectorizer().fit(train_x)
train_tfid_csr = vectorizer.transform(train_x)
test_tfid_csr = vectorizer.transform(test_x)

# change Tfid vectors from scipy.sparse.csr.csr_matrix to tf.sparse.SparseTensor
train_tfid_st = convert_sparse_matrix_to_sparse_tensor(train_tfid_csr)
test_tfid_st = convert_sparse_matrix_to_sparse_tensor(test_tfid_csr)

# change Tfid vecotors from tf.sparse.SparseTensor to tf.Tensor
train_tfid_dense = tf.sparse.to_dense(train_tfid_st)
test_tfid_dense = tf.sparse.to_dense(test_tfid_st)

# # get Tfid vector vocab size
# vocab_size_tfid = train_vectors.shape[1]
# print(train_vectors.shape,test_vectors.shape)

# Training Model 1

In [0]:
embedding_dim = 100

def network():
    model = Sequential()
    model.add(Dense(8192, input_dim=35822, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(11, activation='relu'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [0]:
estimator = KerasClassifier(build_fn=network, epochs=20, batch_size=5, verbose=0)

In [14]:
import time

start_time = time.time()
estimator.fit(train_tfid_st, train_y)
print(f"Training time {time.time()-start_time:.0f}s")
test_pred = estimator.predict(test_tfid_st)
metrics.confusion_matrix(test_y, test_pred, categories)

Training time 894s
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[  0,   0,   0,   0,   0,   0,   3,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,  15,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,  13,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   2,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,  48,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,  14,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 266,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,  69,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   3,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   7,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,  60,   0,   0,   0,   0]])

In [16]:
print(metrics.accuracy_score(test_y, test_pred))
print(metrics.precision_score(test_y, test_pred, average="micro"))
print(metrics.recall_score(test_y, test_pred, average="micro"))

0.532
0.532
0.532


# Training Model 2

In [0]:
embedding_dim = 10

def network():
    global embedding_dim, vocab_size_tfid
    model = Sequential()
    model.add(Embedding(vocab_size_tfid,
                        embedding_dim,
                        input_length=vocab_size_tfid))
    # (batch, vocab_size) -> (batch * vocab_size * embedding_dim)
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(64, activation='relu'))
    # (batch, vocab_size)
    model.add(Dense(11, activation='relu'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model


In [15]:
import time

# get Tfid vector vocab size
vocab_size_tfid = train_tfid_dense.shape[1]

estimator = KerasClassifier(build_fn=network, epochs=20, batch_size=5, verbose=0)
start_time = time.time()
estimator.fit(train_tfid_dense, train_y)
print(f"Training time {time.time()-start_time:.0f}s")
test_pred = estimator.predict(test_tfid_dense)
metrics.confusion_matrix(test_y, test_pred, categories)

Training time 1738s
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[  0,   0,   0,   0,   0,   0,   0,   3,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,  15,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,  13,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   2,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,  48,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,  14,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0, 266,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,  69,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   3,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   7,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,  60,   0,   0,   0]])

# Training Model 3

In [16]:
embedding_dim = 10

def network():
    global embedding_dim, vocab_size
    model = Sequential()
    model.add(Embedding(vocab_size,
                        embedding_dim,
                        input_length=vocab_size))
    # (batch, vocab_size) -> (batch * vocab_size * embedding_dim)
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    # (batch, vocab_size)
    model.add(Dense(11, activation='relu'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

import time

# get count vector vocab size
vocab_size = train_count_dense.shape[1]

estimator = KerasClassifier(build_fn=network, epochs=20, batch_size=5, verbose=0)
start_time = time.time()
estimator.fit(train_count_dense, train_y)
print(f"Training time {time.time()-start_time:.0f}s")
test_pred = estimator.predict(test_count_dense)
metrics.confusion_matrix(test_y, test_pred, categories)

Training time 537s
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[  2,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0],
       [ 10,   0,   0,   0,   0,   0,   0,   5,   0,   0,   0],
       [ 11,   0,   0,   0,   0,   0,   0,   2,   0,   0,   0],
       [  2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  1,   0,   0,   0,   0,   0,   0,  47,   0,   0,   0],
       [ 11,   0,   0,   0,   0,   0,   0,   3,   0,   0,   0],
       [129,   0,   0,   0,   0,   0,   0, 137,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,  69,   0,   0,   0],
       [  3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  1,   0,   0,   0,   0,   0,   0,   6,   0,   0,   0],
       [ 15,   0,   0,   0,   0,   0,   0,  45,   0,   0,   0]])

In [19]:
def network():
    global vocab_size
    model = Sequential()
    model.add(Dense(8192, input_dim=vocab_size, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(11, activation='relu'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

estimator = KerasClassifier(build_fn=network, epochs=20, batch_size=5, verbose=0)

import time


# get count vector vocab size
vocab_size = train_count_dense.shape[1]

start_time = time.time()
estimator.fit(train_count_st, train_y)
print(f"Training time {time.time()-start_time:.0f}s")
test_pred = estimator.predict(test_count_st)
print(metrics.confusion_matrix(test_y, test_pred, categories))

print(metrics.accuracy_score(test_y, test_pred))
print(metrics.precision_score(test_y, test_pred, average="micro"))
print(metrics.recall_score(test_y, test_pred, average="micro"))

Training time 887s
[[  0   0  14   0   0   0   0   0   1   0   0]
 [  0   0   7   0   0   0   0   0   0   0   0]
 [  0   0 239   3   0   0   0  21   3   0   0]
 [  0   0   4  16   0   1   0  27   0   0   0]
 [  0   0  14   0   0   0   0   0   0   0   0]
 [  0   0   3   0   0   0   0   0   0   0   0]
 [  0   0   2   0   0   0   0   0   0   0   0]
 [  0   0   3  16   0   0   0  50   0   0   0]
 [  0   0   1   0   0   0   0   0  59   0   0]
 [  0   0   3   0   0   0   0   0   0   0   0]
 [  0   0  13   0   0   0   0   0   0   0   0]]
0.728
0.728
0.728
